**420-A52-SF - Algorithmes d'apprentissage supervisé - Automne 2022 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2022 Mikaël Swawola
<br/>
![Travaux Pratiques - Régression linéaire multiple](static/02-A1-banner.png)
<br/>
**Objectif:** cette séance de travaux pratique consiste en la mise en oeuvre sous forme de code vectorisé de l'**algorithme du gradient en régression linéaire multiple**. Le jeu de données utilisé sera la version complète du jeu de données *Advertising* et devra être **mis à l'échelle**

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

### 0 - Chargement des bibliothèques

In [2]:
# Manipulation de données
import numpy as np
import pandas as pd
from collections import defaultdict

# Visualisation de données
import matplotlib.pyplot as plt
import seaborn as sns

# Outils divers
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm

In [3]:
# Configuration de la visualisation
sns.set(style="darkgrid", rc={'figure.figsize':(11.7,8.27)})

### 1 - Lecture du jeu de données advertising

**Exercice 1-1**: à l'aide de la bibiothèques *pandas*, lire le fichier `advertising-multivariate.csv`

In [4]:
# Compléter le code ci-dessous ~ 1 ligne
df = pd.read_csv('../../data/advertising-multivariate.csv', usecols=['TV','radio','newspaper','sales'])

**Exercice 1-2**: à l'aide de la fonction `head()`, visualiser les premières lignes de la trame de données. Quelle sera la taille du vecteur de paramètres $\theta$ ?

In [5]:
# Compléter le code ci-dessous ~ 1 ligne
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


### 2 - Mise à l'échelle des données

**Exercice 2**: Standardiser les données.<br/>
Note: Il n'est pas nécéssaire de standardiser la variable de sortie, mais vous pouvez le faire à des fins de simplification

In [6]:
# Compléter le code ci-dessous ~ 1 ligne
df_norm = (df - df.mean(axis=0))/df.std(axis=0)
df_norm.head()

,TV,radio,newspaper,sales
0,0.967425,0.979066,1.774493,1.548168
1,-1.194379,1.080097,0.667903,-0.694304
2,-1.512360,1.524637,1.779084,-0.905135
3,0.051919,1.214806,1.283185,0.858177
4,0.393196,-0.839507,1.278593,-0.215143


### 3 - Préparation de la structure de données

**Exercice 3**: Construire la matrice des prédicteurs X sans oublier d'ajouter une colonne représentant $x_0$

In [7]:
# Compléter le code ci-dessous ~ 5 lignes
x0 = np.ones(shape=(200))
x1 = df_norm['TV'].values
x2 = df_norm['radio'].values
x3 = df_norm['newspaper'].values
X = np.array((x0,x1,x2,x3))

y = df['sales'].values.reshape(-1,1) # Nous gardons ici les valeurs non standardisées

<strong style='color: green'>TEST - Le code ci-dessous vous permet de tester la forme de `X`. Le `assert` ne doit pas renvoyer d'exception</strong>

In [8]:
assert X.shape == (4,200)

### 4 - Définition du modèle

**Exercice 4**: compléter la fonction ci-dessous représentant le modèle de régression linéaire multiple (hypothèse)

Pour rappel, le modèle de régression multiple est

$h_{\theta}(x)=\theta_{0}x_0 + \theta_{1}x_1 + \cdots + \theta_{n}x_n = \theta^TX$

In [9]:
def hypothesis(x, theta):
    assert x.shape[0] == theta.shape[0]
    # Compléter le code ~ 1 ligne
    h = np.dot(theta.T, x)
    return h

<strong style='color: green'>TEST - Le code ci-dessous vous permet de tester votre fonction `hypothesis`. Le `assert` ne doit pas renvoyer d'exception</strong>

In [10]:
x_test = np.array([[1,1],[3,4],[2,2],[1,-1]])
theta_test = np.array([1,2,2,4]).reshape(-1,1)
hypothesis(x_test, theta_test)
assert np.array_equal(hypothesis(x_test,theta_test), np.array([[15,9]]))

### 5 - Fonction de coût

**Exercice 5**: compléter la fonction ci-dessous permettant le calcul du coût (fonction de coût)

Pour rappel, la fonction de coût en régression linéaire multiple s'exprime sous la forme

$J(\theta)= \frac{1}{2m}\sum\limits_{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})^{2}=\frac{1}{2m}(y-X^t\theta)^T\times(y-X^t\theta)$

Remarque: comme le montre l'équation ci-dessus, il exite deux méthodes pour calculer la fonction de coût. Choisissez celle qui vous convient.<br/><em>Optionnel: faites l'autre méthode</em>

In [11]:
def cost_function(x,y, theta):
    # Compléter le code ~ 1-4 lignes
    error = hypothesis(x, theta) - y.T
    squared_error = error**2
    sse = np.sum(squared_error)
    cost = (1/(2*y.shape[0])) * sse
    return cost

<strong style='color: green'>TEST - Le code ci-dessous permet de tester la fonction `cost_function`. Celle-ci doit retourner un `numpy.float64`, c'est-à-dire un nombre et non tableau (array). Le `assert` ne doit pas renvoyer d'exception et le résultat attendu est ~ 94.92</strong>

In [12]:
theta_test = np.array([1,2,2,4]).reshape(-1,1)
cost = cost_function(X,y,theta_test)
assert type(cost) == np.float64
cost

94.91820816343483

In [13]:
def cost_function(x,y, theta):
    # Compléter le code ~ 1-4 lignes
    error = (y.reshape(-1,1) - np.dot(X.T,theta))
    sse = np.dot(error.T, error)
    cost = (1/(2*y.shape[0])) * sse.squeeze()
    return cost

<strong style='color: green'>TEST - Le code ci-dessous permet de tester la fonction `cost_function`. Celle-ci doit retourner un `numpy.float64`, c'est-à-dire un nombre et non tableau (array). Le `assert` ne doit pas renvoyer d'exception et le résultat attendu est ~ 94.92</strong>

In [14]:
theta_test = np.array([1,2,2,4]).reshape(-1,1)
cost = cost_function(X,y,theta_test)
assert type(cost) == np.float64
cost

94.91820816343485

### 6 - Algorithme du gradient

**Exercice 6**: Compléter l'algorithme du gradient ci-dessous. Choisir le vecteur $\theta$ initial, la valeurs du **pas** ($\alpha$) et le **nombre d'itérations**. Un test de convergence ne sera pas utilisé ici.

$
\text{Répéter pendant n_iterations}
\{\\
\theta_{j}:= \theta_{j} - \alpha\frac{1}{m}\sum\limits_{i=1}^{m}(h_{\theta}(x^{(i)})-y^{(i)})\times x_{j}^{(i)}\quad\forall j
\\
\}
$

Ou sous forme vectorisée:

$
\text{Répéter pendant n_iterations}
\{\\
\theta:= \theta - \alpha\frac{1}{m}(\theta^TX-y)\times X
\\
\}
$

<strong>Vous êtes vivement encouragés à utiliser la forme vectorisée. Celle-ci est de toute façon plus simple à coder que la version non vectorisée !</strong>

In [15]:
theta = np.zeros(shape = (X.shape[0],1))
alpha = 0.000066
n_iterations = 650000

m = y.shape[0]

history = defaultdict(list)

for i in tqdm(range(0, n_iterations)):
    
    # Compléter le code ~ 2 lignes
    d_theta = np.dot((np.dot(theta.T,X) - y.T), X.T) / m
    theta = theta - (alpha * d_theta.T)
    
    # Sauvegarde des valeurs intermédiaires de theta et du coût
    if i%50 == 0:
        cost = cost_function(X, y, theta)
        history['theta_0'].append(theta[0])
        history['theta_1'].append(theta[1])
        history['theta_2'].append(theta[2])
        history['theta_3'].append(theta[3])
        history['cost'].append(cost)
        
print(f'Theta = {theta}')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 650000/650000 [00:05<00:00, 115564.81it/s]

Theta = [[14.0225    ]
 [ 3.92908869]
 [ 2.79906919]
 [-0.02259517]]


Les valeurs des paramètres $\theta_j$ devraient approcher

```[[14.0225    ]
 [ 3.92908869]
 [ 2.79906919]
 [-0.02259517]]```

**Version (partiellement) non vectorisée**

In [16]:
theta = np.zeros(shape = (X.shape[0],1))
alpha = 0.000066
n_iterations = 650000
m = y.shape[0]

history = defaultdict(list)

for i in tqdm(range(0, n_iterations)):
    
    d_theta = np.zeros(shape=(X.shape[0],1))
    for j in range(0,X.shape[0]):
        d_theta[j] = np.sum((hypothesis(X, theta) - y)*X[j,:]) / m
    theta = theta - (alpha * d_theta)
    
    # Sauvegarde des valeurs intermédiaires de theta et du coût
    if i%50 == 0:
        cost = cost_function(X, y, theta)
        history['theta_0'].append(theta[0])
        history['theta_1'].append(theta[1])
        history['theta_2'].append(theta[2])
        history['theta_3'].append(theta[3])
        history['cost'].append(cost)
        
print(f'Theta = {theta}')

  4%|████████▏                                                                                                                                                                                     | 27846/650000 [00:10<03:48, 2727.15it/s]


KeyboardInterrupt: 

### 7 - Interprétation des paramètres

**Exercice 7**: Interpréter les paramètres obtenus

### Fin du TP